# Bioinformatics 101

In this notebook, we will manipulate gene expression data.

In [16]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

Importing libraries:

In [8]:
suppressWarnings(library(dplyr))
suppressWarnings(library(tidyverse))
suppressWarnings(library(GEOquery))

Read in the data

In [9]:
dat <- read.csv(file = '../data/GSE183947_fpkm.csv')

In [10]:
dim(dat)

[1] 20246    61

In [11]:
head(dat)

,X,CA.102548,CA.104338,CA.105094,CA.109745,CA.1906415,CA.1912627,CA.1924346,CA.1926760,CA.1927842,⋯,CAP.2040686,CAP.2046297,CAP.2046641,CAP.348981,CAP.354300,CAP.359448,CAP.94377,CAP.98389,CAP.98475,CAP.99145
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,TSPAN6,0.93,1.97,0.00,5.45,4.52,4.75,3.96,3.58,6.41,⋯,6.66,8.35,8.94,6.33,5.94,6.35,3.74,4.84,10.46,4.54
2,TNMD,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.23,0.39,⋯,0.12,0.17,1.08,0.29,0.00,0.07,9.19,1.18,0.09,0.39
3,DPM1,0.00,0.43,0.00,3.43,8.45,8.53,7.80,7.62,6.40,⋯,4.93,7.47,5.72,4.96,9.28,9.15,4.77,3.75,7.31,2.77
4,SCYL3,5.78,5.17,8.76,4.58,7.20,6.03,9.05,5.37,5.92,⋯,8.02,6.00,5.28,4.98,4.45,7.00,4.14,5.51,7.45,2.33
5,C1orf112,2.83,6.26,3.37,6.24,5.16,13.69,6.69,5.28,7.65,⋯,7.91,4.61,8.35,9.84,7.68,5.62,2.81,7.08,7.28,5.39
6,FGR,4.80,1.83,0.00,4.23,15.87,8.56,13.28,12.27,5.58,⋯,6.99,6.16,13.27,19.33,2.89,40.13,10.53,9.00,4.45,11.42


get metadata: to find out which ones are cancer cells or not.

In [12]:
gse <- getGEO(GEO = 'GSE183947', GSEMatrix = TRUE)

Found 1 file(s)

GSE183947_series_matrix.txt.gz

Using locally cached version: /tmp/RtmpMZOTxa/GSE183947_series_matrix.txt.gz

Using locally cached version of GPL11154 found here:
/tmp/RtmpMZOTxa/GPL11154.soft.gz 



In [13]:
gse

$GSE183947_series_matrix.txt.gz
ExpressionSet (storageMode: lockedEnvironment)
assayData: 0 features, 60 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: GSM5574685 GSM5574686 ... GSM5574744 (60 total)
  varLabels: title geo_accession ... tissue:ch1 (41 total)
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
  pubMedIds: 35046993 
Annotation: GPL11154 


In [18]:
metadata <- pData(phenoData(gse[[1]]))
head(metadata, 1)

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,characteristics_ch1.1,characteristics_ch1.2,molecule_ch1,extract_protocol_ch1,extract_protocol_ch1.1,taxid_ch1,description,data_processing,data_processing.1,data_processing.2,data_processing.3,data_processing.4,platform_id,contact_name,contact_institute,contact_address,contact_city,contact_state,contact_zip/postal_code,contact_country,data_row_count,instrument_model,library_selection,library_source,library_strategy,relation,relation.1,supplementary_file_1,donor:ch1,metastasis:ch1,tissue:ch1
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM5574685,tumor rep1,GSM5574685,Public on Sep 15 2021,Sep 11 2021,Sep 15 2021,SRA,1,breast,Homo sapiens,tissue: breast tumor,metastasis: yes,donor: 102548,total RNA,"Total RNA was isolated and purified using TRIzol (Life, cat.265709, CA, USA) following the manufacturer's procedure.","After the quality inspection of Agilent 2100 Bioanalyzer (Agilent, cat.G2939AA, CA, USA) and NanoPhotometer (Implen, cat.N60, Munich, Germany), mRNA with poly(A) is purified from 1μg total RNA using VAHTS mRNA Capture Beads with Oligo (dT) (Vazyme, cat.N401-01, Nanjing, China) through two rounds of purification. Subsequently, mRNA fragment was interrupted using VAHTS Universal V6 RNA-seq Library Prep Kit (Vazyme, cat.NR604, Nanjing, China) under 94℃ 8min and reversed transcription into cDNA which would use to synthesise U-labeled second-stranded DNAs. An A-base was added to the blunt ends of each strand to ligase the indexed adapters which contains a T-base at the tail end. After UDG enzyme treatment of the U-labeled double-strand DNA, size selection was performed with VAHTS DNA Clean Beads (Vazyme, cat.N411, Nanjing, China). Then the ligated products are amplified with PCR by the following conditions: initial denaturation at 98℃ for 5 min; 12-17 cycles of denaturation at 98℃ for 10 sec, annealing at 60℃ for 30 sec, and extension at 72℃ for 30 sec; final extension at 72℃ for 5 min. The average insert size of cDNA library was 280±80 bp. After purification by VAHTS DNA Clean Beads (Vazyme, cat.N411-02, Nanjing, China), quality control of concentration and fragment size is performed by Agilent 2100 Bioanalyzer (Agilent, cat.G2939AA, CA, USA) and Qubit assay tubes (Life, cat. 1604220, CA, USA).",9606,CA.102548,"In order to remove technical sequences, including adapters, polymerase chain reaction (PCR) primers, or fragments thereof, and quality of bases lower than 20, pass filter data of fastq format were processed by Cutadapt (V1.9.1) to be high quality clean data.","Firstly, reference genome sequences and gene model annotation files of relative species were downloaded from genome ENSEMBL website, Tophat2 (v2.1.0) was used to index reference genome sequence. Finally, clean data were aligned to reference genome via software Tophat2 (v2.1.02).","In the beginning transcripts in fasta format are converted from known gff annotation file and indexed properly. Then, with the file as a reference gene file, RSEM (v1.3.1) estimated gene and isoform expression levels from the pair-end clean data.",Genome_build: Homo sapiens (GRCh37),Supplementary_files_format_and_content: fpkm.csv: Comma-separated text file includes FPKM values for each Sample.,GPL11154,"Jianhua,,Xu",Department of Laboratory Medicine Shunde Hospital of Guangzhou University of Chinese Medicine,"No. 12, Jinsha Avenue, Shunde District, Foshan, Guangdong, P.R. China",Fushan,Guangdong,528300,China,0,Illumina HiSeq 2000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN21395376,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX12143676,NONE,102548,yes,breast tumor


In [19]:
metadata.subset <- select(metadata, c(1,10,11,17))

In [28]:
metadata.subset[1,]

,title,characteristics_ch1,characteristics_ch1.1,description
,<chr>,<chr>,<chr>,<chr>
GSM5574685,tumor rep1,tissue: breast tumor,metastasis: yes,CA.102548


In [39]:
metadata.modified <- metadata %>%
    select(1,10,11,17) %>%
    rename(tissue = characteristics_ch1) %>%
    rename(metastatis = characteristics_ch1.1) %>%
    mutate(tissue = gsub("tissue: ", "", tissue)) %>%
    mutate(metastatis = gsub("metastasis: ", "", metastatis))

In [42]:
metadata.modified[1:3,]

,title,tissue,metastatis,description
,<chr>,<chr>,<chr>,<chr>
GSM5574685,tumor rep1,breast tumor,yes,CA.102548
GSM5574686,tumor rep2,breast tumor,yes,CA.104338
GSM5574687,tumor rep3,breast tumor,yes,CA.105094


In [44]:
head(dat)

,X,CA.102548,CA.104338,CA.105094,CA.109745,CA.1906415,CA.1912627,CA.1924346,CA.1926760,CA.1927842,CA.1933414,CA.1940640,CA.2004407,CA.2005288,CA.2006047,CA.2008260,CA.2009329,CA.2009381,CA.2009850,CA.2017611,CA.2039179,CA.2040686,CA.2045012,CA.2046297,CA.348981,⋯,CAP.1912627,CAP.1924346,CAP.1926760,CAP.1927842,CAP.1933414,CAP.1940640,CAP.2004407,CAP.2005288,CAP.2006047,CAP.2008260,CAP.2009329,CAP.2009381,CAP.2009850,CAP.2017611,CAP.2039179,CAP.2040686,CAP.2046297,CAP.2046641,CAP.348981,CAP.354300,CAP.359448,CAP.94377,CAP.98389,CAP.98475,CAP.99145
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,TSPAN6,0.93,1.97,0.00,5.45,4.52,4.75,3.96,3.58,6.41,11.89,5.55,3.82,7.66,11.48,4.32,14.44,10.26,6.51,5.86,7.13,9.19,6.65,10.13,7.07,⋯,0.00,10.45,7.10,5.21,5.61,6.20,6.79,11.24,15.73,12.05,11.64,10.16,9.13,6.87,8.71,6.66,8.35,8.94,6.33,5.94,6.35,3.74,4.84,10.46,4.54
2,TNMD,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.23,0.39,0.44,0.36,0.09,0.07,1.14,0.00,0.50,0.21,0.20,0.10,0.18,0.20,0.00,0.00,0.00,⋯,0.00,0.00,0.20,11.43,0.26,3.20,0.06,0.08,0.00,0.00,0.59,0.63,0.49,0.00,0.23,0.12,0.17,1.08,0.29,0.00,0.07,9.19,1.18,0.09,0.39
3,DPM1,0.00,0.43,0.00,3.43,8.45,8.53,7.80,7.62,6.40,6.09,6.62,5.51,9.62,6.77,13.13,9.71,9.01,7.71,18.03,8.32,5.96,9.21,8.60,2.89,⋯,0.00,7.85,4.19,0.44,9.96,0.00,4.33,4.13,0.00,0.75,5.72,3.64,6.13,19.28,3.01,4.93,7.47,5.72,4.96,9.28,9.15,4.77,3.75,7.31,2.77
4,SCYL3,5.78,5.17,8.76,4.58,7.20,6.03,9.05,5.37,5.92,12.45,10.44,8.81,9.04,9.36,5.91,4.58,8.10,13.55,3.66,4.39,7.46,9.98,15.58,8.54,⋯,8.71,4.24,6.07,5.47,4.30,4.90,4.07,7.45,0.23,5.05,5.28,5.90,7.97,2.59,6.57,8.02,6.00,5.28,4.98,4.45,7.00,4.14,5.51,7.45,2.33
5,C1orf112,2.83,6.26,3.37,6.24,5.16,13.69,6.69,5.28,7.65,13.71,13.97,7.30,16.13,14.37,10.71,9.87,8.51,16.54,10.59,5.37,8.59,13.38,15.07,8.33,⋯,5.43,3.90,6.29,14.61,3.81,15.07,3.89,4.58,0.39,14.69,6.95,8.67,10.39,19.20,5.42,7.91,4.61,8.35,9.84,7.68,5.62,2.81,7.08,7.28,5.39
6,FGR,4.80,1.83,0.00,4.23,15.87,8.56,13.28,12.27,5.58,7.03,5.04,12.83,9.56,7.57,9.77,15.40,11.62,8.90,1.87,8.93,9.04,8.44,5.54,15.79,⋯,0.00,6.62,5.24,2.76,7.56,2.19,9.92,8.48,0.00,11.08,14.92,4.35,7.21,1.87,4.48,6.99,6.16,13.27,19.33,2.89,40.13,10.53,9.00,4.45,11.42


Reshaping data into a long format from the wide format.

In [46]:
dat.long <- dat %>%
    rename(gene = X) %>%
    gather(key = 'samples', value = 'FPKM', -gene)

In [48]:
head(dat.long)

,gene,samples,FPKM
,<chr>,<chr>,<dbl>
1,TSPAN6,CA.102548,0.93
2,TNMD,CA.102548,0.00
3,DPM1,CA.102548,0.00
4,SCYL3,CA.102548,5.78
5,C1orf112,CA.102548,2.83
6,FGR,CA.102548,4.80


In [49]:
dim(dat.long)

[1] 1214760       3

join dataframes = dat.long + metadata.modified

In [50]:
dat.long %>%
    left_join(., metadata.modified, by = c('samples' = 'description')) %>%
    head()

,gene,samples,FPKM,title,tissue,metastatis
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,TSPAN6,CA.102548,0.93,tumor rep1,breast tumor,yes
2,TNMD,CA.102548,0.00,tumor rep1,breast tumor,yes
3,DPM1,CA.102548,0.00,tumor rep1,breast tumor,yes
4,SCYL3,CA.102548,5.78,tumor rep1,breast tumor,yes
5,C1orf112,CA.102548,2.83,tumor rep1,breast tumor,yes
6,FGR,CA.102548,4.80,tumor rep1,breast tumor,yes


In [51]:
dat.long <- dat.long %>%
    left_join(., metadata.modified, by = c('samples' = 'description'))

In [54]:
head(dat.long)

,gene,samples,FPKM,title,tissue,metastatis
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,TSPAN6,CA.102548,0.93,tumor rep1,breast tumor,yes
2,TNMD,CA.102548,0.00,tumor rep1,breast tumor,yes
3,DPM1,CA.102548,0.00,tumor rep1,breast tumor,yes
4,SCYL3,CA.102548,5.78,tumor rep1,breast tumor,yes
5,C1orf112,CA.102548,2.83,tumor rep1,breast tumor,yes
6,FGR,CA.102548,4.80,tumor rep1,breast tumor,yes


Explore data

In [63]:
dat.long %>%
    filter(gene == 'BRCA1' | gene == 'BRCA2') %>%
    group_by(gene, tissue) %>%
    summarise(mean_FPKM = mean(FPKM),
              median_FPKM = median(FPKM)) %>%
    arrange(-mean_FPKM) %>%
    head()

`summarise()` has grouped output by 'gene'. You can override using the
`.groups` argument.


gene,tissue,mean_FPKM,median_FPKM
<chr>,<chr>,<dbl>,<dbl>
BRCA1,breast tumor,10.042333,6.965
BRCA1,normal breast tissue,7.698667,6.450
BRCA2,normal breast tissue,3.047333,1.250
BRCA2,breast tumor,2.044000,1.600
